In this example, we will show how to load numpy array data into the new 
TensorFlow 'Dataset' API. The Dataset API implements an optimized data pipeline
with queues, that make data processing and training faster (especially on GPU). 

In [0]:
from __future__ import division, print_function, absolute_import

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/tmp/data", one_hot=True)

W0826 11:57:03.120323 140053471946624 deprecation.py:323] From <ipython-input-2-4b314b129b7a>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0826 11:57:03.122106 140053471946624 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0826 11:57:03.125782 140053471946624 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instruction

Extracting /tmp/data/train-images-idx3-ubyte.gz


W0826 11:57:03.440669 140053471946624 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0826 11:57:03.444736 140053471946624 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0826 11:57:03.497658 140053471946624 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: __init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a f

Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
#training parameters
learning_rate=0.001
num_steps=1000
batch_size=128
display_step=100

#network parameters
n_input=784
n_classes=10
dropout=0.75

sess = tf.Session()

#create dataset tensor from iamges and labels
dataset=tf.data.Dataset.from_tensor_slices(
(mnist.train.images, mnist.train.labels))

#automatically refill data queue when empty
dataset=dataset.repeat()

#create batches of data
dataset=dataset.batch(batch_size)

#prefetch data for faster consumption
dataset=dataset.prefetch(batch_size)

#create an iterator over dataset
iterator=dataset.make_initializable_iterator()

#initialize iterator
sess.run(iterator.initializer)

#neural net input (images,labels)
X, Y=iterator.get_next()

W0826 11:57:04.306494 140053471946624 deprecation.py:323] From <ipython-input-3-c7f6b7fd6ae6>:27: make_initializable_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.


In [0]:
#create model
def conv_net(x, n_classes, dropout, reuse, is_training):
  #define scope for reusing variables
  with tf.variable_scope('ConvNet', reuse=reuse):
    x=tf.reshape(x, shape=[-1, 28,28,1])
    
    #convolution layer with 32 filters and kernel size of 5
    conv1=tf.layers.conv2d(x,32,5,activation=tf.nn.relu)
    #max pooling (down-sampling) with strides of 2 and kernel size of 2
    conv1=tf.layers.max_pooling2d(conv1,2,2)
    
    #convolution layer with 64 filters and kernel size of 3
    conv2=tf.layers.conv2d(conv1, 64,3,activation =tf.nn.relu)
    #max pooling (down-sampling) with strides of 2 and kernel size of 2
    conv2=tf.layers.max_pooling2d(conv2,2,2)
    
    #flatten the data to 1-D vector for fully connected layer
    fc1=tf.contrib.layers.flatten(conv2)
    
    #fully connected layer (in contrib folder for now)
    fc1=tf.layers.dense(fc1,1024)
    #apply dropout (if is_training is False, dropout is not applied)
    fc1=tf.layers.dropout(fc1, rate=dropout, training=is_training)
    
    #output layer, class prediction
    out=tf.layers.dense(fc1, n_classes)
    #because 'softmax_cross_entropy_with_logits' already apply softmax,
    #we only apply softmax to testing network
    out=tf.nn.softmax(out) if not is_training else out
    
  return out

In [5]:
#because dropout have different behaviour at training and prediction time,
#we need to create 2 distinct computation graphs that share same weights

#create graph for training

logits_train = conv_net(X, n_classes, dropout, reuse=False, is_training=True)
#create another graph for testing that reuse the same weights, but has 
#differet behaviour for 'dropout' (not applied)
logits_test=conv_net(X,n_classes, dropout, reuse=True, is_training=False)

#define loss and optimizer (with train logits, for dropout to take effect)
loss_op=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
logits=logits_train, labels=Y))
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op=optimizer.minimize(loss_op)

#evaluate model (with etst logits, for dropout to be disabled)
correct_pred=tf.equal(tf.argmax(logits_test,1), tf.argmax(Y,1))
accuracy=tf.reduce_mean(tf.cast(correct_pred, tf.float32))

W0826 11:57:05.110433 140053471946624 deprecation.py:323] From <ipython-input-4-56f7cd988130>:7: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0826 11:57:05.235747 140053471946624 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0826 11:57:05.454293 140053471946624 deprecation.py:323] From <ipython-input-4-56f7cd988130>:9: max_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
W0826 11:57:05.607307 140053471946624 deprecation.py:323] From /usr/local/lib

In [6]:
#initialize variables
init=tf.global_variables_initializer()

#run initializer
sess.run(init)

#training cycle
for step in range(1, num_steps+1):
  #run optimization
  sess.run(train_op)
  
  if step%display_step==0 or step==1:
    #calculate btch loss and accuracy
    #not that ths consume a new batch of data
    loss, acc=sess.run([loss_op, accuracy])
    print("Step",step, "Loss", loss, "Accuracy", acc)

print("optimization finished")

Step 1 Loss 2.2318397 Accuracy 0.203125
Step 100 Loss 0.15196376 Accuracy 0.953125
Step 200 Loss 0.086668625 Accuracy 0.96875
Step 300 Loss 0.085339144 Accuracy 0.9921875
Step 400 Loss 0.052631423 Accuracy 0.9765625
Step 500 Loss 0.14079179 Accuracy 0.96875
Step 600 Loss 0.05404635 Accuracy 0.96875
Step 700 Loss 0.017887535 Accuracy 1.0
Step 800 Loss 0.0202116 Accuracy 1.0
Step 900 Loss 0.059633136 Accuracy 0.984375
Step 1000 Loss 0.0026733442 Accuracy 1.0
optimization finished
